# Overview

ZnTrack provides an easy to use framework for creating and tracking Experiments.
It is build on top of [DVC](https://dvc.org/), a tool for Version Controlling Machine Learning Projects.
For an introduction we highly recommend reading the [DVC Getting Started](https://dvc.org/doc/start).
Besides version controlled data management, DVC provides method for building a dependency graph, tracking parameters, comparing metrics, reducing computational overhead and queueing multiple runs.

**Why do I want to use ZnTrack?**

Whilst DVC provides all this functionality it is designed to be programming language independent.
This can require writing custom python scripts, reading and writing config files and managing depencencies.

ZnTrack is designed to make these steps as easy and well integrated with Python as possible.
In comparison  to the DVC backbone, it is aimed directly at python developeres and therefore allows a highly adapted and optimized interface.

In [1]:
from zntrack import config

# When using ZnTrack we can write our code inside a Jupyter notebook.
# We can make use of this functionality by setting the `nb_name` config as follows:
config.nb_name = "01_Intro.ipynb"

In [2]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

Working with DVC requires a GIT and DVC repository which we can set up easily:

In [3]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmptn0i8j7r/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


## Nodes
DVC organizes its pipeline in multiple stages.
A stage can be created by inheriting from `zntrack.Node` and implementing a `run()` method.

The `run()` method will later be executed by our pipeline manager (e.g. `dvc repro`).
To make dvc familiar with our new Node we can write the input script with the `write_graph()` command.

Let's start with an example of creating a random integer between 0 and a parameterized maximum value.

In [4]:
from zntrack import Node, zn
from random import randrange


class RandomNumber(Node):
    number = zn.outs()
    maximum = zn.params()

    def run(self):
        self.number = randrange(self.maximum)

Notice that ZnTrack will automatically generate an `__init__` for all `zn.params`.
When writing a custom `__init__` it is important to add `super().__init__(*args, **kwargs)` for ZnTrack to work.
```python
    def __init__(self, maximum=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.maximum = maximum
```


We can now call `run_and_save()` to create our random number.

In [5]:
RandomNumber(maximum=512).run_and_save()

or we can use `write_graph()` to create the DVC stage and let DVC run the commands for us.
By default ZnTrack will only build the graph without running it. We can change that by passing `no_exec=True`.
You can also pass other commands like `always_changed=True, external=True` to the `write_graph()` command.

In [6]:
RandomNumber(maximum=512).write_graph(no_exec=False)

2022-01-21 16:35:09,327 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-01-21 16:35:09,328 (WARNING): Converting 01_Intro.ipynb to file RandomNumber.py


[NbConvertApp] Converting notebook 01_Intro.ipynb to script


2022-01-21 16:35:12,397 (WARNING): --- Writing new DVC file! ---
2022-01-21 16:35:12,398 (WARNING): You will not be able to see the stdout/stderr of the process in real time!


[NbConvertApp] Writing 3241 bytes to 01_Intro.py


2022-01-21 16:35:15,953 (INFO): Running stage 'RandomNumber':
> python3 -c "from src.RandomNumber import RandomNumber; RandomNumber.load(name='RandomNumber').run_and_save()" 
Creating 'dvc.yaml'
Adding stage 'RandomNumber' in 'dvc.yaml'
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run:

    git add dvc.lock dvc.yaml nodes/RandomNumber/.gitignore

To enable auto staging, run:

	dvc config core.autostage true



To gain access to the results we can load the Node via the classmethod `load()` and look at the number attribute.

In [7]:
RandomNumber.load().number

125

In [8]:
temp_dir.cleanup()